In [ ]:
!pip install snscrape

In [ ]:
import pandas as pd

df = pd.read_csv("/content/new.csv", on_bad_lines='skip', quoting=3)

df.head()


,index,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,...,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,0,681448150,FALSE,finalized,3,2/25/15 5:24,neutral,1,NaN,NaN,...,NaN,cairdin,NaN,0.0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.700000e+17,NaN,Eastern Time (US & Canada)
1,1,681448153,FALSE,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0,...,NaN,jnardino,NaN,0.0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.700000e+17,NaN,Pacific Time (US & Canada)
2,2,681448156,FALSE,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,...,NaN,yvonnalynn,NaN,0.0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.700000e+17,Lets Play,Central Time (US & Canada)
3,3,681448158,FALSE,finalized,3,2/25/15 3:05,negative,1,Bad Flight,0.7033,...,NaN,jnardino,NaN,0.0,"""@VirginAmerica it's really aggressive to blas...",NaN,2/24/15 11:15,5.700000e+17,NaN,Pacific Time (US & Canada)
4,4,681448159,FALSE,finalized,3,2/25/15 5:50,negative,1,Can't Tell,1,...,NaN,jnardino,NaN,0.0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/15 11:14,5.700000e+17,NaN,Pacific Time (US & Canada)


In [ ]:
df.columns


Index(['index', '_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'airline_sentiment',
       'airline_sentiment:confidence', 'negativereason',
       'negativereason:confidence', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [ ]:
import re

# Define cleaning function
def clean_tweet(tweet):
    tweet = str(tweet).lower()  # convert to lowercase
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)  # remove URLs
    tweet = re.sub(r"@\w+|#\w+", "", tweet)  # remove mentions and hashtags
    tweet = re.sub(r"[^\w\s]", "", tweet)  # remove punctuation
    tweet = re.sub(r"\d+", "", tweet)  # remove numbers
    tweet = tweet.strip()  # remove leading/trailing spaces
    return tweet

# Apply function to create cleaned column
df["clean_text"] = df["text"].apply(clean_tweet)

# Show original and cleaned text side by side
df[["text", "clean_text"]].head()


,text,clean_text
0,@VirginAmerica What @dhepburn said.,what said
1,@VirginAmerica plus you've added commercials t...,plus youve added commercials to the experience...
2,@VirginAmerica I didn't today... Must mean I n...,i didnt today must mean i need to take another...
3,"""@VirginAmerica it's really aggressive to blas...",its really aggressive to blast obnoxious enter...
4,@VirginAmerica and it's a really big bad thing...,and its a really big bad thing about it


In [ ]:
!pip install textblob
from textblob import TextBlob


In [ ]:
df["sentiment_score"] = df["clean_text"].apply(lambda x: TextBlob(x).sentiment.polarity)


In [ ]:
def get_sentiment_label(score):
    if score > 0.05:
        return "Positive"
    elif score < -0.05:
        return "Negative"
    else:
        return "Neutral"

df["sentiment_label"] = df["sentiment_score"].apply(get_sentiment_label)

# Preview final result
df[["clean_text", "sentiment_score", "sentiment_label"]].head()


,clean_text,sentiment_score,sentiment_label
0,what said,0.00000,Neutral
1,plus youve added commercials to the experience...,0.00000,Neutral
2,i didnt today must mean i need to take another...,-0.31250,Negative
3,its really aggressive to blast obnoxious enter...,0.00625,Neutral
4,and its a really big bad thing about it,-0.35000,Negative


In [ ]:
final_df = df[["text", "clean_text", "sentiment_score", "sentiment_label"]]


In [ ]:
final_df.to_csv("tweet_sentiment_output.csv", index=False)


In [ ]:
from google.colab import files
files.download("tweet_sentiment_output.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>